## Retriever and Chain

In [1]:
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader("DNA_Bacteria_Signature using Genomic DNA Sequencing.pdf")
docs = loader.load()
docs

[Document(metadata={'producer': 'www.ilovepdf.com', 'creator': 'Microsoft® Word 2016', 'creationdate': '2025-09-03T07:41:29+00:00', 'author': 'python-docx', 'moddate': '2025-09-03T07:41:29+00:00', 'source': 'DNA_Bacteria_Signature using Genomic DNA Sequencing.pdf', 'total_pages': 12, 'page': 0, 'page_label': '1'}, page_content='Detection of Bacterial Signatures in Genomic DNA Sequences using Hybrid \nML-DL Pipeline \n \n \n \n \n \n \n \n \n \nAbstract \nProper identification and classification of bacterial life \nin DNA sequences is one of the central roles of \nbioinformatics with general implications for clinical \ndiagnosis, typing of pathogens, characterization of \nmicrobiome, and genetic research. A fast pace of \nadvancements in sequencing technologies has amassed \nvast volumes of genomic data, necessitating robust \ncomputational approaches to analyze and interpret \nbiological patterns effectively.Although earlier \nmachine learning (ML) and deep learning (DL) \nmodels have 

In [2]:

from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
text_splitter.split_documents(docs)[:5]

[Document(metadata={'producer': 'www.ilovepdf.com', 'creator': 'Microsoft® Word 2016', 'creationdate': '2025-09-03T07:41:29+00:00', 'author': 'python-docx', 'moddate': '2025-09-03T07:41:29+00:00', 'source': 'DNA_Bacteria_Signature using Genomic DNA Sequencing.pdf', 'total_pages': 12, 'page': 0, 'page_label': '1'}, page_content='Detection of Bacterial Signatures in Genomic DNA Sequences using Hybrid \nML-DL Pipeline \n \n \n \n \n \n \n \n \n \nAbstract \nProper identification and classification of bacterial life \nin DNA sequences is one of the central roles of \nbioinformatics with general implications for clinical \ndiagnosis, typing of pathogens, characterization of \nmicrobiome, and genetic research. A fast pace of \nadvancements in sequencing technologies has amassed \nvast volumes of genomic data, necessitating robust \ncomputational approaches to analyze and interpret \nbiological patterns effectively.Although earlier \nmachine learning (ML) and deep learning (DL) \nmodels have 

In [3]:

documents=text_splitter.split_documents(docs)
documents

[Document(metadata={'producer': 'www.ilovepdf.com', 'creator': 'Microsoft® Word 2016', 'creationdate': '2025-09-03T07:41:29+00:00', 'author': 'python-docx', 'moddate': '2025-09-03T07:41:29+00:00', 'source': 'DNA_Bacteria_Signature using Genomic DNA Sequencing.pdf', 'total_pages': 12, 'page': 0, 'page_label': '1'}, page_content='Detection of Bacterial Signatures in Genomic DNA Sequences using Hybrid \nML-DL Pipeline \n \n \n \n \n \n \n \n \n \nAbstract \nProper identification and classification of bacterial life \nin DNA sequences is one of the central roles of \nbioinformatics with general implications for clinical \ndiagnosis, typing of pathogens, characterization of \nmicrobiome, and genetic research. A fast pace of \nadvancements in sequencing technologies has amassed \nvast volumes of genomic data, necessitating robust \ncomputational approaches to analyze and interpret \nbiological patterns effectively.Although earlier \nmachine learning (ML) and deep learning (DL) \nmodels have 

In [4]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import FAISS
from dotenv import load_dotenv
import os
# Load API key from .env
load_dotenv()
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")

# Initialize Gemini embeddings
embeddings = GoogleGenerativeAIEmbeddings(
    model="models/gemini-embedding-001"   # ✅ Gemini embedding model
)
db=FAISS.from_documents(documents,embeddings)



d:\Data Science\LangChain\GenAi_BotGemma\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
query="What are the ML algorithms used in this "
result=db.similarity_search(query)
result[0].page_content

'4.3.3 XGBoost Classifier \nXGBoost or Extreme Gradient Boosting is a very \nfast regularized boosting algorithm that performs \nvery well on structured data problems. XGBoost \nbuilds decision trees sequentially, so one tree \nshould help minimize the errors from the \nprevious trees. It also has regularization terms in \nthe objective function to avoid overfitting.  \nWe chose XGBoost because it can also \nefficiently work on sparse high-dimensional \ndata, and it also has built-in parallelization in the \ntraining which makes computation faster. Some \nof the parameters we used were max_depth=6, \nn_estimators=100 and learning_rate=0.1. \nTable 3. Performance metrics of XGB \nClass  Precision  Recall  F1-\nScore  \nSupport  \n1 0.99 0.98 0.98 9,987 \n \nFigure 5: Confusion matrix for XGBoost \nclassifier. \n \n \n4.3.4 Model comparison and takeaways \nAll models performed nearly the same with 98% \noverall accuracy, suggesting biological inspired \nfeatures are good predictors. Alth

In [ ]:
# from langchain_ollama import OllamaLLM

# llm= OllamaLLM(model="gemma")
# llm

In [7]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""
Answer the following question based only on the provided conext
Think step by step before providing a detailed answer
I'll tip you $1000 if the user finds the answer helpful
<context>{context}</context>
Question: {input}

""")

In [10]:
## Chain
from langchain.chains.combine_documents import create_stuff_documents_chain

document_Chain=create_stuff_documents_chain(llm,prompt)

In [11]:
"""
Retrievers: A retriever is an interface that returns documents given
 an unstructured query. It is more general than a vector store.
 A retriever does not need to be able to store documents, only to 
 return (or retrieve) them. Vector stores can be used as the backbone
 of a retriever, but there are other types of retrievers as well. 
 https://python.langchain.com/docs/modules/data_connection/retrievers/   
"""

retriever=db.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'GoogleGenerativeAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001D78D36FE00>, search_kwargs={})

In [12]:
from langchain.chains import create_retrieval_chain
retriever_chain=create_retrieval_chain(retriever,document_Chain)

In [11]:
# response=retriever_chain.invoke({"input":""" The CNN model extracts local patterns and
# distant dependencies in sequences such that the model
# learns deep representations"""})

In [12]:
# response['answer']

In [9]:
# using gemini
from langchain_google_genai import ChatGoogleGenerativeAI

import os
from dotenv import load_dotenv

# Load .env file
load_dotenv()

# ✅ Correct env variable
api_key = os.getenv("GOOGLE_API_KEY")
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-pro",
    google_api_key=api_key,   
    temperature=0.7
)

In [14]:
response=retriever_chain.invoke({"input":""" The CNN model extracts local patterns and
distant dependencies in sequences such that the model
learns deep representations"""})

In [16]:
response

{'input': ' The CNN model extracts local patterns and\ndistant dependencies in sequences such that the model\nlearns deep representations',
 'context': [Document(id='ce74a3d0-fd2e-4948-948f-bc337db29b12', metadata={'producer': 'www.ilovepdf.com', 'creator': 'Microsoft® Word 2016', 'creationdate': '2025-09-03T07:41:29+00:00', 'author': 'python-docx', 'moddate': '2025-09-03T07:41:29+00:00', 'source': 'DNA_Bacteria_Signature using Genomic DNA Sequencing.pdf', 'total_pages': 12, 'page': 5, 'page_label': '6'}, page_content='4.3 Deep Learning with 1D Convolutional \nNeural Network (CNN) \nTo eliminate manual feature extraction and allow \nthe model to learn directly from the genomic \nsequences in its raw form, a 1D Convolutional \nNeural Network (CNN) was used. This structure \nis particularly appropriate for sequential data and \nhas performed exceedingly well in applications \ninvolving natural language and biological \nsequences. The key advantage of CNNs is that \nthey perform localized